## Objective

Now that I know that my submodels perform a bit better than the model on the entire dataset, I can take advantage of the submodels and find the coefficients that are most associated with response variable.

In [1]:
CATEGORY_GROUPS_IN_QUESTION = [['Pick up Dead Animal', 'Animal Generic Request'],
['Abandoned Vehicles', 'Abandoned Bicycle'],
['Rodent Activity',	'Bed Bugs', 'Mice Infestation - Residential'],
['Sidewalk Repair', 'Sidewalk Repair (Make Safe)'],
['Needle Pickup'],
['Unsatisfactory Living Conditions', 'Poor Conditions of Property', 'Unsanitary Conditions - Establishment', 'Illegal Occupancy', 'Heat - ],Excessive  Insufficient'],
['Request for Pothole Repair'],
['Graffiti Removal']]

## Objective

Does this have statistically significant coefs, assuming homoskedacity and a linear predictor-response relationship and Normalized residuals and imperfect collinearity?

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
from tqdm import tqdm

from utilities import remove_one_feature

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))
from helper_functions import dummify_cols_and_baselines, make_alphas, remove_outliers_by_type, adjusted_r2, transform_school, get_vifs

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

In [5]:
df_orig = transform_school(df_orig)
df_orig.shape

../preprocessing/helper_functions.py:58: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  df.school = df.school.str.extract(r'(\d\d?)').astype(int)


(516406, 40)

## Filterering by `TYPE`

In [6]:
i = 4
print CATEGORY_GROUPS_IN_QUESTION[i]
df_orig = df_orig[df_orig.TYPE.isin(CATEGORY_GROUPS_IN_QUESTION[i])]
df_orig.shape

['Needle Pickup']


(1683, 40)

## Removing outliers

A standard procedure is to remove values further than 3 standard deviations from the mean. Since I have so many low values and some very high values, I anecdotally think that the low values are very likely to be true, and the high values not so much.

So, I will remove values further than 3 SDs from the median, by type.

Ideally, I would take into account the time dimension. I would like to do so given more time.

In [7]:
df_outliers_removed = remove_outliers_by_type(df_orig, y_col='COMPLETION_HOURS_LOG_10')
df_outliers_removed.shape

(1683, 40)

## Remove `TYPE` col

In [8]:
df_outliers_removed.drop('TYPE', axis=1, inplace=True)

## Choosing columns

In [ ]:
['race_black',
 'race_asian',
 'race_other',
 'poverty_pop_below_poverty_level',
 'earned_income_per_capita',
 'poverty_pop_w_public_assistance',
 'poverty_pop_w_food_stamps',
 'poverty_pop_w_ssi',
 'school',
 'school_std_dev',
 'housing_std_dev',
 'bedroom',
 'bedroom_std_dev',
 'value_std_dev',
 'rent_std_dev',
 'income',
 'income_std_dev',
 'is_description',
 'Property_Type_Address']

In [65]:
cols_orig_dataset = ['COMPLETION_HOURS_LOG_10', 'SubmittedPhoto', 'Source']
cols_census = [
     'housing',
     'value',
     'rent',
]
cols_engineered = ['queue_wk_open']

In [66]:
df = df_outliers_removed[cols_orig_dataset + cols_census + cols_engineered]
df.shape

(1683, 7)

## Removing NAs for cols like `school_std_dev`

In [67]:
aa = df.isnull().any().reset_index()
nas = aa[aa[0] == True]['index']
print nas

Series([], Name: index, dtype: object)


In [68]:
# this is a bad temporary band-aid
df = df.dropna(subset=nas.tolist())
df.shape

(1683, 7)

## Dummify

In [69]:
cols_to_dummify = [i for i in df.dtypes[df.dtypes == object].index if i != 'TYPE']
cols_to_dummify

['Source', 'housing']

In [70]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify, chosen_col_i=2)

Constituent Call is baseline 0 2
own is baseline 1 2


In [53]:
df_dummified.shape

(1683, 7)

## Checking for multicollinearity

In [96]:
df_dummified.head(1).T

,905400
COMPLETION_HOURS_LOG_10,0.0124857
SubmittedPhoto,False
poverty_pop_below_poverty_level,0.262473
bedroom,2
queue_wk,12873
queue_wk_open,1
is_description,True


In [16]:
get_vifs(df_dummified.drop(['SubmittedPhoto', 'is_description'], axis=1), 'COMPLETION_HOURS_LOG_10')

poverty_pop_w_food_stamps          5.627159
poverty_pop_below_poverty_level    5.140319
earned_income_per_capita           4.206161
income                             3.604180
race_black                         3.046973
school_std_dev                     2.545789
race_hispanic                      2.517314
poverty_pop_w_ssi                  2.462153
income_std_dev                     2.388088
rent                               2.360315
school                             2.313099
rent_std_dev                       2.235951
housing_std_dev                    2.223495
poverty_pop_w_public_assistance    2.018333
race_asian                         1.907403
value                              1.906657
value_std_dev                      1.900194
bedroom                            1.832490
bedroom_std_dev                    1.821070
queue_wk                           1.431254
queue_wk_open                      1.353225
race_other                         1.239467
Property_Type_Address           

## Running model

In [17]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import string
from StringIO import StringIO


In [18]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('COMPLETION_HOURS_LOG_10', axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

## Use LassoCV to find col subsets

In [20]:
pipe = make_pipeline(StandardScaler(), LassoCV())
cv = ShuffleSplit(X_train.shape[0], n_iter=1, test_size=0.2, random_state=300)

In [26]:
params = {'lassocv__alphas': make_alphas(-3, 0)}
model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=cv, verbose=0)
model.fit(X_train, y_train)
pd.DataFrame(model.cv_results_).T.iloc[2:5]

,0,1,2,3,4,5,6
mean_test_score,0.248127,0.25034,0.252115,0.256669,0.254728,0.162828,-0.00151683
mean_train_score,0.277228,0.276695,0.272925,0.265274,0.243276,0.147442,0
param_lassocv__alphas,[0.001],[0.003],[0.01],[0.03],[0.1],[0.3],[1.0]


In [27]:
model.best_params_

{'lassocv__alphas': [0.029999999999999999]}

In [28]:
'{} cols go to zero out of {}'.format(
    len(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0]),
    len(X_train.columns)
)

'19 cols go to zero out of 26'

In [29]:
cols_zero = list(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0])
cols_zero

['race_black',
 'race_asian',
 'race_other',
 'poverty_pop_below_poverty_level',
 'earned_income_per_capita',
 'poverty_pop_w_public_assistance',
 'poverty_pop_w_food_stamps',
 'poverty_pop_w_ssi',
 'school',
 'school_std_dev',
 'housing_std_dev',
 'bedroom',
 'bedroom_std_dev',
 'value_std_dev',
 'rent_std_dev',
 'income',
 'income_std_dev',
 'is_description',
 'Property_Type_Address']

## Use subsetted cols to run lin reg

In [71]:
df_dummified.columns = [col.translate(None, string.punctuation).replace(' ', '') if col != 'COMPLETION_HOURS_LOG_10' else col for col in df_dummified.columns]

In [72]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop(['COMPLETION_HOURS_LOG_10'], axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

In [73]:
col_list = ' + '.join(df_dummified.drop(['COMPLETION_HOURS_LOG_10'], axis=1).columns)

est = smf.ols(
    'COMPLETION_HOURS_LOG_10 ~ {}'.format(col_list), 
    pd.concat([X_train, y_train], axis=1)).fit()

In [40]:
est.summary().tables[0]

Dep. Variable:,COMPLETION_HOURS_LOG_10,R-squared:,0.285
Model:,OLS,Adj. R-squared:,0.279
Method:,Least Squares,F-statistic:,53.14
Date:,"Thu, 23 Feb 2017",Prob (F-statistic):,4.09e-90
Time:,08:21:29,Log-Likelihood:,-1790.7
No. Observations:,1346,AIC:,3603.
Df Residuals:,1335,BIC:,3661.
Df Model:,10,,
Covariance Type:,nonrobust,,


### Getting adjusted $R^2$ on test set

In [74]:
y_pred = est.predict(X_test)

In [75]:
adjusted_r2(y_test, y_pred, num_features=X_test.shape[1])

0.094542508861745436

In [76]:
mean_squared_error(y_test, y_pred)**0.5

0.98628188906589287

## Interpreting model

Which features are most associated with completion time?

In [77]:
df_results = pd.read_csv(StringIO(est.summary().tables[1].as_csv()), index_col=0).reset_index()
df_results.columns = ['coef_name'] + [i.rstrip().lstrip() for i in df_results.columns][1:]
df_results['coef_abs'] = pd.np.abs(df_results.coef)
df_results.coef_name = df_results.coef_name.map(lambda x: x.strip())
df_results = df_results.sort_values('P>|t|')
df_results.shape

(6, 7)

In [78]:
df_results

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],coef_abs
0,Intercept,3.752000e-01,9.300000e-02,4.056,0.000,0.194 0.557,3.752000e-01
3,rent,2.000000e-04,4.090000e-05,4.105,0.000,8.76e-05 0.000,2.000000e-04
4,queuewkopen,7.000000e-03,1.000000e-03,10.374,0.000,0.006 0.008,7.000000e-03
2,value,-2.611000e-07,8.230000e-08,-3.174,0.002,-4.23e-07 -9.97e-08,2.611000e-07
5,SourceCitizensConnectApp,2.150000e-01,8.100000e-02,2.643,0.008,0.055 0.375,2.150000e-01
1,SubmittedPhoto[T.True],1.737000e-01,7.800000e-02,2.222,0.026,0.020 0.327,1.737000e-01


In [64]:
scores = []

for col in X_train.columns:
    if col != 'Intercept':
        score = remove_one_feature([col], df_dummified)
        scores.append((col, score))
        
sorted(scores, key=lambda x: x[1])[::-1]        

[('SourceCitizensConnectApp', 18),
 ('queuewk', 18),
 ('SubmittedPhoto', 16),
 ('queuewkopen', 15.837),
 ('rent', 14),
 ('value', 11.891)]

## Conclusion

Now that I think about it, Needle Pickup is not a good category to look at. The location of this issue is whereever the needle is, which is ...